In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import json

with open('/content/Streaming_History_Audio_2024-2025_1.json', 'r') as f:
  data = json.load(f)

In [ ]:
import pandas as pd

# Read the JSON file
df = pd.read_json('/content/Streaming_History_Audio_2024-2025_1.json')

# Drop sensitive columns
df = df.drop(columns=['ip_address', 'conn_country'], errors='ignore')

In [ ]:
!pip install spotipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 7.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datetime import datetime

# Load the data (assuming it's already uploaded as in the previous step)
with open('Streaming_History_Audio_2024-2025_1.json', 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data)
df['ts'] = pd.to_datetime(df['ts'])
df['hour'] = df['ts'].dt.hour
df['day_of_week'] = df['ts'].dt.day_name()
df['month'] = df['ts'].dt.strftime('%Y-%m')

# Define skipping heuristic (no API needed)
standard_duration = 180000  # Assume 3-minute average track length
df['is_skipped'] = df.apply(
    lambda row: row['skipped'] or (row['ms_played'] < 0.3 * standard_duration and row['reason_end'] in ['endplay', 'fwdbtn', 'backbtn']),
    axis=1
)

# Skip rate by artist
artist_skips = df.groupby('master_metadata_album_artist_name')['is_skipped'].mean() * 100
artist_skips = artist_skips.sort_values(ascending=False)
print("\nSkip Rate by Artist (%):")
print(artist_skips.head(10))

# Skip rate by track
track_skips = df.groupby('master_metadata_track_name')['is_skipped'].mean() * 100
track_skips = track_skips.sort_values(ascending=False)
print("\nMost Skipped Tracks (%):")
print(track_skips.head(10))


Skip Rate by Artist (%):
master_metadata_album_artist_name
Zeeshan Ali              100.0
Zara Larsson             100.0
ZAYN                     100.0
Yüzyüzeyken Konuşuruz    100.0
Younger Hunger           100.0
Young the Giant          100.0
Yo Yo Honey Singh        100.0
Yasser Desai             100.0
Yashraj                  100.0
YUNGBLUD                 100.0
Name: is_skipped, dtype: float64

Most Skipped Tracks (%):
master_metadata_track_name
Şıkır Şıkır               100.0
vampire                   100.0
things i wish you said    100.0
the perfect pair          100.0
the grudge                100.0
the boy is mine           100.0
tere bina                 100.0
sweat                     100.0
still feel.               100.0
socha na tha              100.0
Name: is_skipped, dtype: float64


In [ ]:
# Listening frequency by hour
hourly_listening = df.groupby('hour').size()
print("\nListening Frequency by Hour:")
print(hourly_listening)

# Skip rate by hour
hourly_skips = df.groupby('hour')['is_skipped'].mean() * 100
print("\nSkip Rate by Hour (%):")
print(hourly_skips)

# Listening frequency by day of the week
daily_listening = df.groupby('day_of_week').size().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)
print("\nListening Frequency by Day of Week:")
print(daily_listening)

# Listening frequency by month
monthly_listening = df.groupby('month').size()
print("\nListening Frequency by Month:")
print(monthly_listening)


Listening Frequency by Hour:
hour
0      18
1      88
2     172
3     340
4     433
5     708
6     683
7     655
8     522
9     650
10    796
11    722
12    691
13    614
14    506
15    427
16    425
17    603
18    751
19    428
20    246
21    146
22     58
23      2
dtype: int64

Skip Rate by Hour (%):
hour
0     16.666667
1     38.636364
2     26.744186
3     37.352941
4     41.108545
5     36.440678
6     35.724744
7     46.106870
8     26.436782
9     33.538462
10    34.296482
11    36.011080
12    40.520984
13    44.462541
14    35.968379
15    39.110070
16    38.352941
17    38.474295
18    46.737683
19    39.719626
20    34.552846
21    14.383562
22     8.620690
23     0.000000
Name: is_skipped, dtype: float64

Listening Frequency by Day of Week:
day_of_week
Monday       1374
Tuesday      1398
Wednesday    1323
Thursday     1245
Friday       1643
Saturday     1957
Sunday       1744
dtype: int64

Listening Frequency by Month:
month
2024-01     478
2024-02     661
2024-03  

In [ ]:
# Most played artists
artist_plays = df.groupby('master_metadata_album_artist_name').size().sort_values(ascending=False)
print("\nMost Played Artists:")
print(artist_plays.head(10))

# Total listening time by artist (in minutes)
artist_time = df.groupby('master_metadata_album_artist_name')['ms_played'].sum() / 60000  # Convert ms to minutes
artist_time = artist_time.sort_values(ascending=False)
print("\nTotal Listening Time by Artist (Minutes):")
print(artist_time.head(10))


Most Played Artists:
master_metadata_album_artist_name
Taylor Swift         3096
Twenty One Pilots    3069
Hasan Raheem          645
Sabrina Carpenter     314
Talwiinder            299
Lana Del Rey          265
Abdul Hannan          261
Umair                 194
Simge                 108
Bayaan                 78
dtype: int64

Total Listening Time by Artist (Minutes):
master_metadata_album_artist_name
Twenty One Pilots    8924.724300
Taylor Swift         8564.503650
Hasan Raheem         1616.642350
Talwiinder            759.367650
Lana Del Rey          695.277750
Sabrina Carpenter     599.366017
Abdul Hannan          539.117867
Umair                 517.062933
Simge                 316.266650
The Neighbourhood     149.467500
Name: ms_played, dtype: float64


In [ ]:
import matplotlib.pyplot as plt

# Visualize top artists
plt.figure(figsize=(10, 6))
artist_plays.head(10).plot(kind='bar')
plt.title('Top 10 Most Played Artists')
plt.xlabel('Artist')
plt.ylabel('Number of Plays')
plt.xticks(rotation=45, ha='right')
plt.savefig('top_artists.png')
plt.close()

In [ ]:
# Save to CSV (no index)
df.to_csv('Streaming_History_Audio_2024-2025_1.csv', index=False)

# Download in Colab
from google.colab import files
files.download('Streaming_History_Audio_2024-2025_1.csv')
